# Introduction

In this project, we'll practice with chunked dataframes and optimize a dataframe's memory usage using Pandas. We'll work with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. If we read in the entire dataset, it consumes about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory and see what we can do.

# Load Data

In [1]:
import pandas as pd
pd.options.display.max_columns = 99

In [2]:
first_five = pd.read_csv('loans_2007.csv', nrows=5)
first_five.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          5 non-null      int64  
 1   member_id                   5 non-null      float64
 2   loan_amnt                   5 non-null      float64
 3   funded_amnt                 5 non-null      float64
 4   funded_amnt_inv             5 non-null      float64
 5   term                        5 non-null      object 
 6   int_rate                    5 non-null      object 
 7   installment                 5 non-null      float64
 8   grade                       5 non-null      object 
 9   sub_grade                   5 non-null      object 
 10  emp_title                   3 non-null      object 
 11  emp_length                  5 non-null      object 
 12  home_ownership              5 non-null      object 
 13  annual_inc                  5 non-null 

In [172]:
first_five

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
first_1000 = pd.read_csv('loans_2007.csv', nrows=1000)
first_1000.memory_usage(deep=True).sum()/(1024**2)

1.5502548217773438

The first 1000 rows occupy around 1.6MB of memory. We are searching for a rough number of rows to read that occupy around half of the available memory (ie 10MB / 2 == 5 MB).

In [4]:
memory_used = first_1000.memory_usage(deep=True).sum()/(1024**2)
rows = 1000
while memory_used <= 5:
    df = pd.read_csv('loans_2007.csv', nrows=rows)
    memory_used = df.memory_usage(deep=True).sum()/(1024**2)
    print(rows, memory_used)
    rows += 500

1000 1.5502548217773438
1500 2.3244361877441406
2000 3.099329948425293
2500 3.874464988708496
3000 4.649059295654297
3500 5.42338752746582


It looks like we should use around 3000 rows per chunk. Let's verify that none of the chunks using the `read_csv` iterable object go over 5MB.

In [55]:
memory_used = []
df_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in df_chunks:
    memory_used.append(chunk.memory_usage(deep=True).sum()/(1024**2))

for m in memory_used:
    print(m, m > 5)

4.649059295654297 False
4.644805908203125 False
4.646563529968262 False
4.647915840148926 False
4.644108772277832 False
4.645991325378418 False
4.644582748413086 False
4.646951675415039 False
4.645077705383301 False
4.64512825012207 False
4.657840728759766 False
4.656707763671875 False
4.663515090942383 False
4.896956443786621 False
0.880854606628418 False


In [57]:
sum(memory_used)

66.21605968475342

# Data Exploration

Here, we'll answer some fundamental questions about the data set (by chunk):

- How many columns have a numeric type? How many have a string type?
- How many unique values are there in each string column?
- Which columns are dates?
- Which float columns have no missing values and could be candidates for conversion to the integer type?

In [6]:
df_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
data_types = []
for chunk in df_chunks:
    data_types.append(chunk.dtypes.value_counts())
    
data_types = pd.concat(data_types)
data_types = data_types.groupby(data_types.index).sum()
data_types

int64       13
float64    450
object     317
dtype: int64

It looks like around 40% (317/(13+450+317)) of the columns are strings.

In [7]:
df_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in df_chunks:
    object_cols = chunk.select_dtypes('object').columns
    
object_cols

Index(['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status',
       'last_pymnt_d', 'last_credit_pull_d', 'application_type'],
      dtype='object')

In [173]:
first_five[object_cols]

,id,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type
0,1077501,36 months,10.65%,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,Jan-1985,83.7%,f,Jan-2015,Jun-2016,INDIVIDUAL
1,1077430,60 months,15.27%,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,Apr-1999,9.4%,f,Apr-2013,Sep-2013,INDIVIDUAL
2,1077175,36 months,15.96%,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,Nov-2001,98.5%,f,Jun-2014,Jun-2016,INDIVIDUAL
3,1076863,36 months,13.49%,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,Feb-1996,21%,f,Jan-2015,Apr-2016,INDIVIDUAL
4,1075358,60 months,12.69%,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,Jan-1996,53.9%,f,Jun-2016,Jun-2016,INDIVIDUAL


The following columns should be converted to numeric: id, term, int_rate, revol_util. The following columns should be converted to datetime: earliest_cr_line, last_pymnt_d, last_credit_pull_d.

In [174]:
df_chunks = pd.read_csv('loans_2007.csv', chunksize=3000)
object_col_values = {}
for chunk in df_chunks:
    for col in object_cols:
        vals = chunk[col].value_counts()
        if col in object_col_values:
            object_col_values[col].append(vals)
        else:
            object_col_values[col] = [vals]
            
for col in object_col_values:
    object_col_values[col] = pd.concat(object_col_values[col])
    object_col_values[col] = object_col_values[col].groupby(object_col_values[col].index).sum()
    

unique_items = {key: len(object_col_values[key]) for key in object_col_values}
dict(sorted(unique_items.items(), key=lambda x:x[1]))

{'initial_list_status': 1,
 'application_type': 1,
 'term': 2,
 'pymnt_plan': 2,
 'verification_status': 3,
 'home_ownership': 5,
 'grade': 7,
 'loan_status': 9,
 'emp_length': 11,
 'purpose': 14,
 'sub_grade': 35,
 'addr_state': 50,
 'issue_d': 55,
 'last_pymnt_d': 103,
 'last_credit_pull_d': 108,
 'int_rate': 394,
 'earliest_cr_line': 530,
 'zip_code': 837,
 'revol_util': 1119,
 'title': 21264,
 'emp_title': 30658,
 'id': 42538}

From this list, we can drop `initial_list_status` and `application_type`, and convert everything up to `addr_state` to factors. We will also drop `id` because we don't need it.

# Convert and Clean Data

We will add the following to the batch import step:
- Drop missing values for `member_id`
- Add the date columns as dates
- Convert the category columns to categories
- Clean up `term`, `int_rate`, and `revol_util` and convert to numeric
- Check if any of the float columns can be converted to integer without losing data.

In [167]:
to_date_cols = ['earliest_cr_line', 'issue_d', 'last_pymnt_d', 'last_credit_pull_d']
to_category_cols = ['pymnt_plan', 'verification_status', 'home_ownership', 'grade','loan_status','emp_length', 'purpose', 'sub_grade', 'addr_state']

In [168]:
memory_used_part_two = []
df_chunks = pd.read_csv('loans_2007.csv', dtype={col: 'category' for col in to_category_cols}, chunksize=3000, parse_dates=to_date_cols)
for chunk in df_chunks:
    chunk = chunk[~chunk['member_id'].isna()]
    float_cols = chunk.select_dtypes('float').columns
    for float_col in float_cols:
        if chunk[float_col].apply(lambda x: x.is_integer()).all():
            chunk[float_col] = chunk[float_col].astype('int')
    chunk['term'] = pd.to_numeric(chunk['term'].str.replace('[a-zA-Z ]', '', regex=True))
    chunk['int_rate'] = pd.to_numeric(chunk['int_rate'].str.replace('%', '', regex=True))
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'].str.replace('%', '', regex=True))
    memory_used_part_two.append(chunk.memory_usage(deep=True).sum()/(1024**2))

In [169]:
chunk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 42000 to 42535
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id                          536 non-null    object        
 1   member_id                   536 non-null    int64         
 2   loan_amnt                   536 non-null    int64         
 3   funded_amnt                 536 non-null    int64         
 4   funded_amnt_inv             536 non-null    float64       
 5   term                        536 non-null    int64         
 6   int_rate                    536 non-null    float64       
 7   installment                 536 non-null    float64       
 8   grade                       536 non-null    category      
 9   sub_grade                   536 non-null    category      
 10  emp_title                   499 non-null    object        
 11  emp_length                  536 non-null    category

In [170]:
sum(memory_used_part_two)

27.247037887573242

In [171]:
sum(memory_used) - sum(memory_used_part_two)

38.969021797180176

We gained almost 40MB in space!